In [1]:
## CSV Files downloaded from https://www.data.gouv.fr/fr/datasets/repertoire-national-des-associations/  Fichier RNA Waldec du 01 Mars 2022

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext(appName="LIst of associations")

22/03/17 22:42:52 WARN Utils: Your hostname, DESKTOP-3481LPC resolves to a loopback address: 127.0.1.1; using 172.21.11.8 instead (on interface eth0)
22/03/17 22:42:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/17 22:42:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [68]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

file_location = "/mnt/d/Projects/mongulu/acu-te/tests/data/rna_waldec_20220301/"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", "ISO-8859-1") \
  .load(file_location)

df.show(5)
df.count()

+----------+------+-----+------+-------+----------+----------+----------+----------+------+----------+--------------------+--------------------+--------------------+-------------+-------------+---------------+------------+---------------+-------------+----------------+------------+--------------+---------------+--------------------+--------------+--------------+---------------+--------------------+------------+---------------+--------------------+---------+------------+-------+--------+--------------------+--------+-------------------+
|        id| id_ex|siret|rup_mi|gestion|date_creat|date_decla|date_publi|date_disso|nature|groupement|               titre|         titre_court|               objet|objet_social1|objet_social2|adrs_complement|adrs_numvoie|adrs_repetition|adrs_typevoie|    adrs_libvoie|adrs_distrib|adrs_codeinsee|adrs_codepostal|     adrs_libcommune|adrg_declarant|adrg_complemid|adrg_complemgeo|        adrg_libvoie|adrg_distrib|adrg_codepostal|       adrg_achemine|adrg_pa

2119460

In [48]:
df.columns

['id',
 'id_ex',
 'siret',
 'rup_mi',
 'gestion',
 'date_creat',
 'date_decla',
 'date_publi',
 'date_disso',
 'nature',
 'groupement',
 'titre',
 'titre_court',
 'objet',
 'objet_social1',
 'objet_social2',
 'adrs_complement',
 'adrs_numvoie',
 'adrs_repetition',
 'adrs_typevoie',
 'adrs_libvoie',
 'adrs_distrib',
 'adrs_codeinsee',
 'adrs_codepostal',
 'adrs_libcommune',
 'adrg_declarant',
 'adrg_complemid',
 'adrg_complemgeo',
 'adrg_libvoie',
 'adrg_distrib',
 'adrg_codepostal',
 'adrg_achemine',
 'adrg_pays',
 'dir_civilite',
 'siteweb',
 'publiweb',
 'observation',
 'position',
 'maj_time']

In [ ]:
from pyspark.sql.functions import col,lower,upper

# fix enconding with https://github.com/rspeer/python-ftfy in case not well written
df2 = df.withColumn("titre",upper(col("titre")))\
  .withColumn("objet",lower(col("objet")))\
  .select("titre", "objet", "objet_social1","objet_social2","adrs_numvoie","adrs_typevoie","adrs_libvoie","adrs_codepostal","adrs_libcommune",  "siteweb") \
  .filter(df.titre.contains("CAMEROUN") | df.objet.contains("cameroun") )

df2.show()
df2.count()

In [71]:
# Print files to CSV
df2.toPandas().to_csv("ref-rna-real-mars-2022.csv")